## PCX - Phuture Crypto Market Index - Methodology Template

## Configuring packages

In [1]:
import pandas as pd
import decouple
import sys
sys.path.append('../')
from methodology import MethodologyBase
from pycoingecko import CoinGeckoAPI
key =  decouple.config("CG_KEY")
cg = CoinGeckoAPI(api_key=key)

## Index Variables

In [3]:
min_mcap =  5e8
min_weight = 0.005
max_weight = 0.3
circ_supply_threshold = 0.2
liveness_threshold = 180 # In days
liquidity_consistency = 90 # In days
coingecko_category = None

# Calculating max slippage based on liquidity threshold
liq = 5*1e6
liq_os = liq / 2 # one-side liquidity 
trade_value_tiny = 1e2
trade_value = 1e5
max_slippage = (liq_os + trade_value_tiny) / (liq_os + trade_value) - 1
# round to the nearest .01
max_slippage = round(max_slippage, 2)

# Dataframes to remove
stablecoins = pd.DataFrame(cg.get_coins_markets('usd',category='stablecoins')).set_index('id')
ctokens = pd.DataFrame(cg.get_coins_markets('usd',category='compound-tokens')).set_index('id')
atokens = pd.DataFrame(cg.get_coins_markets('usd',category='aave-tokens')).set_index('id')

methodology_class = MethodologyBase(min_mcap,
                                min_weight,
                                max_weight,
                                255,
                                circ_supply_threshold,
                                liveness_threshold,
                                liquidity_consistency,
                                max_slippage,
                                coingecko_category)

## Token inclusion criteria

1. **Project and token characteristics**
    1. The project’s token should have been listed on CoinGecko with pricing data spanning at least 6 months prior to the date of inclusion in the index.
    1. The project should have an active community.
    1. No rebasing or deflationary tokens.
    1. Synthetic tokens which derive their value from external price feeds are not permissible. 
    1. The project’s token must not have the ability to pause token transfers.
    1. The project must have a minimum circulating market cap of $50mm.
   <br>
1. **Pricing requirmeents**
    1. Token should have a reliable price feed from one of our authorised providers. 
    <br>
1. **Token Supply requirements**
    1. The project's token must have a circulating supply greater than 20% of the max supply. In cases where a token does not have a max supply, the minting mechanics would need to be assessed.
    <br>
1. **Liquidity Requirements**
    1. The token must be listed on a supported exchange.
    1. The price should experience no more than 12% price impact when executing a $100k trade.
    <br>
1. **Security Requirements**
    1. The project must have been audited by smart contract security professionals with the audit report(s) publicly available. Alternatively, the protocol must have been operating long enough to create a consensus about its safety in the decentralised finance community.
1. **Weighting requirements**
    1. The maximum weight any one token can have is 30%.
    1. All excess weight is proportionally redistributed to all uncapped tokens. After this has been completed, if another token now exceeds the 30% threshold the excess will be redistributed to the remaining uncapped tokens. This process will occur iteratively until there are no tokens that exceed the maximum weight.
    1. Any asset with a weight below 0.5% will be removed from the index.


## Run Methodology

In [3]:

results,liquidity = methodology_class.main(
    df_to_remove=[stablecoins,ctokens,atokens],
    ids_to_replace=[['bitcoin'],['wrapped-bitcoin']],
    remove_category_assets=['staked-ether','wrapped-bitcoin','rocket-pool-eth','frax-ether','staked-frax-ether']
    )
liquidity

Excluding mantle, pricing data available only for 144 < 180 days
Excluding beam-2, pricing data available only for 43 < 180 days
0x0000000000000000000000000000000000001010 polygon-pos
0xbec243c995409e6520d7c41e404da5deba4b209b avalanche
0x925206b8a707096ed26ae47c84747fe0bb734f59 ethereum
0x19de6b897ed14a376dda0fe53a5420d2ac828a28 ethereum
 ethereum
0xe5b49820e5a1063f6f4ddf851327b5e8b2301048 polygon-pos
0xa0b862f60edef4452f25b4160f177db44deb6cf1 arbitrum-one
0x7205705771547cf79201111b4bd8aaf29467b9ec polygon-pos


,spot price,delivery price,slippage,blockchain
id,,,,
lido-dao,3.943512e-01,0.398119,0.009554,ethereum
gala,3.131393e+01,31.522209,0.006651,ethereum
illuvium,9.601992e-03,0.009649,0.004920,ethereum
uniswap,1.584753e-01,0.159075,0.003782,ethereum
ethereum,4.228357e-04,0.000424,0.003667,ethereum
maker,6.887598e-04,0.000689,0.000187,ethereum
wrapped-bitcoin,2.308000e-05,0.000023,-0.000433,arbitrum-one
binancecoin,4.299895e-03,0.004298,-0.000437,binance-smart-chain
matic-network,1.194052e+00,1.193271,-0.000654,polygon-pos


## Show Results

In [4]:
results

,name,market_cap,weight,weight_converted,address,blockchain_with_highest_liq
id,,,,,,
wrapped-bitcoin,Bitcoin,846196298746,0.312048,80,0x2f2a2543b76a4166549f7aab2e75bef0aefc5b0f,arbitrum-one
ethereum,Ethereum,282793209979,0.312048,80,ETH,ethereum
binancecoin,BNB,35764191184,0.144836,37,BNB,binance-smart-chain
avalanche-2,Avalanche,9599742301,0.038877,10,AVAX,avalanche
chainlink,Chainlink,8693998141,0.035209,9,0x514910771af9ca656af840dff83e8264ecf986ca,ethereum
matic-network,Polygon,7762389484,0.031436,9,0x0000000000000000000000000000000000001010,polygon-pos
the-open-network,Toncoin,7643113863,0.030953,8,0x582d872a1b094fc48f5de31d3b73f2d9be47def1,ethereum
shiba-inu,Shiba Inu,5754084763,0.023303,6,0x95ad61b0a150d79219dcf64e1e6cc01f0b64c4ce,ethereum
uniswap,Uniswap,4703564566,0.019048,5,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984,ethereum
